# Homework 1

1. Write all code in the notebook.
1. Write all text in the notebook. You can use MathJax to insert math or generic Markdown to insert figures (it's unlikely you'll need the latter). 
1. **Execute** the notebook and **save** the results.
1. You can save the notebook as a PDF as well.


In [ ]:
%mavenRepo snapshots https://oss.sonatype.org/content/repositories/snapshots/

%maven ai.djl:api:0.17.0
%maven org.slf4j:slf4j-api:1.7.26
%maven org.slf4j:slf4j-simple:1.7.26

%maven ai.djl.mxnet:mxnet-engine:0.17.0

In [ ]:
import ai.djl.Device;
import ai.djl.ndarray.NDManager;
import ai.djl.ndarray.NDArray;
import ai.djl.ndarray.LazyNDArray;
import ai.djl.ndarray.index.NDIndex;
import ai.djl.ndarray.types.Shape;

// You have to use the MXNet engine for Q1 since
// it is currently the only engine in DJL
// to support Lazy NDArrays
NDManager manager = NDManager.newBaseManager(Device.cpu(), "MXNet");

In [ ]:
%load ../../utils/StopWatch

## 1. Speedtest for vectorization

Your goal is to measure the speed of linear algebra operations for different levels of vectorization. You need to use the LazyNDArrays `waitToRead()` function on the output to ensure that the result is computed completely, since NDArray uses asynchronous computation. Please see https://javadoc.io/doc/ai.djl/api/latest/ai/djl/ndarray/LazyNDArray.html for details. 

Hint: The MXNet NDArray manager utilizes MxNDArrays which implement the LazyNDArray class. 
You must cast the return NDArray to LazyNDArray to be able to call
the `waitToRead()` function.

1. Construct two matrices $A$ and $B$ with Gaussian random entries of size $4096 \times 4096$. 
1. Compute $C = A B$ using matrix-matrix operations and report the time. 
1. Compute $C = A B$, treating $A$ as a matrix but computing the result for each column of $B$ one at a time. Report the time.
1. Compute $C = A B$, treating $A$ and $B$ as collections of vectors. Report the time.
1. Bonus question - what changes if you execute this on a GPU?

Note: The vector by vector calculation will likely take over 1.5 hours on a modern cpu.

In [ ]:
var A = manager.randomNormal(new Shape(4096, 4096));
var B = manager.randomNormal(new Shape(4096, 4096));
var sw = new StopWatch();
var C = (LazyNDArray) A.dot(B);
C.waitToRead();
System.out.printf("Matrix by matrix: %f seconds\n", sw.stop());

C = (LazyNDArray) manager.zeros(new Shape(4096, 4096));
sw.start();
for (int i = 0; i < 4096; i++) {
    C.set(new NDIndex(":").addIndices(i),
            A.dot(B.get(new NDIndex(":").addIndices(i))));
}
C.waitToRead();
System.out.printf("Matrix by vector: %f seconds\n", sw.stop());

// Note: this will likely take over 1.5 hrs on a standard cpu.
C = (LazyNDArray) manager.zeros(new Shape(4096, 4096));
sw.start();
for (int i = 0; i < 4096; i++) {
    for (int j = 0; j < 4096; j++) {
        C.set(new NDIndex(i, j),
                A.get(new NDIndex(i).addIndices(":"))
                        .dot(B.get(new NDIndex(":").addIndices(j))));
    }
}
C.waitToRead();
System.out.printf("Vector by vector: %f seconds\n", sw.stop());

## 2. Semidefinite Matrices

Assume that $A \in \mathbb{R}^{m \times n}$ is an arbitrary matrix and that $D \in \mathbb{R}^{n \times n}$ is a diagonal matrix with nonnegative entries. 

1. Prove that $B = A D A^\top$ is a positive semidefinite matrix. 
1. When would it be useful to work with $B$ and when is it better to use $A$ and $D$?

1. $B$ is positive semidefinite if \forall $x \in \mathbb{R}^m, x^TBx \geq 0$. Then let $x \in \mathbb{R}^m$ and let $y = A^Tx$. Then
$$x^TBx = x^TADA^T = (A^Tx)^TDA^Tx=y^TDy=\sum_{1}^{n}d_iy_i^2 \geq 0$$
1. It would be more useful to work with $B$ if $m << n$ as matrix multiplication for instance for two arbitrary matrices $X$ and $Y$ with dimensions $a$ by $b$ and $b$ by $c$ is a runtime of $O(abc)$. 
If we multiply $B$ by a matrix $C$ that is $m$ by $k$, then $BC$ is computed in $O(m^2k).$ 
This matrix multiplication with $ADA^TC$ is $O(2mnk + n^2k)$. 
Thus if $m << n$, then it would be more efficient to use $B$ and if $n >> m$, then it would be better to use $A$ and $D$.

## 3. MXNet on GPUs

1. Install GPU drivers (if needed)
1. Install MXNet on a GPU instance
1. Display `!nvidia-smi`
1. Create a $2 \times 2$ matrix on the GPU and print it. See https://d2l.djl.ai/chapter_deep-learning-computation/use-gpu.html for details.

In [ ]:
var managerGPU = NDManager.newBaseManager(Device.gpu());
var x = managerGPU.ones(new Shape(2, 2));
x;

## 4. Memory efficient computation

We want to compute $C \leftarrow A \cdot B + C$, where $A, B$ and $C$ are all matrices. Implement this in the most memory efficient manner. Pay attention to the following two things:

1. Do not allocate new memory for the new value of $C$.
1. Do not allocate new memory for intermediate results if possible.

In [ ]:
var A = manager.randomNormal(new Shape(100, 100));
var B = manager.randomNormal(new Shape(100, 100));
var C = manager.randomNormal(new Shape(100, 100));

C.addi(A.dot(B));

## 5. Broadcast Operations

In order to perform polynomial fitting we want to compute a design matrix $A$ with 

$$A_{ij} = x_i^j$$

Our goal is to implement this **without a single for loop** entirely using vectorization and broadcast. Here $1 \leq j \leq 20$ and $x = \{-10, -9.9, \ldots 10\}$. Implement code that generates such a matrix.

In [ ]:
var x = manager.arange(-10, 10.1f, 0.1f).reshape(new Shape(201, 1));
var j = manager.arange(1, 21, 1f).reshape(new Shape(1, 20));
x.pow(j);